# ANALISE DO SPOTIFY

### Imports

In [1]:
import os
import json
import pandas as pd
from typing import Dict, Any

### Metodos

In [2]:
def folder_to_df_merged(folder_path: str) -> Dict[str, pd.DataFrame]:
    result = {}

    for file in os.listdir(folder_path):
        if not file.endswith(".json"):
            continue

        path = os.path.join(folder_path, file)

        with open(path, encoding="utf-8") as f:
            data: Any = json.load(f)

        if isinstance(data, list) and all(isinstance(i, dict) for i in data):
            df = pd.json_normalize(data)

        elif isinstance(data, dict):
            list_keys = [k for k, v in data.items() if isinstance(v, list)]

            if list_keys:
                main_key = list_keys[0]
                df = pd.json_normalize(
                    data,
                    record_path=main_key,
                    meta=[k for k in data.keys() if k != main_key],
                    errors="ignore"
                )
            else:
                df = pd.DataFrame([data])

        else:
            df = pd.DataFrame()

        result[file.replace(".json", "")] = df
    
    return pd.concat(result.values(), ignore_index=True)


In [3]:
df = folder_to_df_merged("../data/raw")
df

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2019-01-28T17:08:51Z,"Android OS 8.0.0 API 26 (samsung, SM-J600GT)",5287,BR,191.183.197.182,Wiggle (feat. Snoop Dogg),Jason Derulo,Tattoos,spotify:track:4B4zDmbHdkXZ1wUJv1yKFy,None,...,None,None,None,playbtn,trackdone,True,False,False,NaN,False
1,2019-01-28T17:08:56Z,"Android OS 8.0.0 API 26 (samsung, SM-J600GT)",4900,BR,191.183.197.182,Summer,Calvin Harris,Motion,spotify:track:6YUTL4dYpB9xZO5qExPf05,None,...,None,None,None,trackdone,trackdone,True,False,False,NaN,False
2,2019-01-28T17:08:58Z,"Android OS 8.0.0 API 26 (samsung, SM-J600GT)",1440,BR,191.183.197.182,Waiting For Love,Avicii,Stories,spotify:track:2P4OICZRVAQcYAV2JReRfj,None,...,None,None,None,trackdone,trackdone,True,False,False,NaN,False
3,2019-01-28T17:08:59Z,"Android OS 8.0.0 API 26 (samsung, SM-J600GT)",463,BR,191.183.197.182,Uptown Funk (feat. Bruno Mars),Mark Ronson,Uptown Special,spotify:track:32OlwWuMpZ6b0aN2RZOeMS,None,...,None,None,None,trackdone,trackdone,True,False,False,NaN,False
4,2019-01-28T17:09:00Z,"Android OS 8.0.0 API 26 (samsung, SM-J600GT)",166,BR,191.183.197.182,This Is How We Do,Katy Perry,PRISM,spotify:track:5vL0yvddknhGj7IrBc6UTj,None,...,None,None,None,trackdone,trackdone,True,False,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198800,2025-05-15T09:43:43Z,android,246733,BR,189.57.250.90,Symphony Of Destruction,Megadeth,Countdown To Extinction,spotify:track:51TG9W3y9qyO8BY5RXKgnZ,None,...,None,None,None,trackdone,trackdone,True,False,False,1.747302e+09,False
198801,2025-05-15T09:48:05Z,android,261040,BR,189.57.250.90,Emerald Sword,Rhapsody,Symphony of Enchanted Lands,spotify:track:5efydZMkYiKSdRnvjCtTqo,None,...,None,None,None,trackdone,trackdone,True,False,False,1.747302e+09,False
198802,2025-05-15T09:52:40Z,windows,145255,BR,170.85.20.181,November Rain,Guns N' Roses,Use Your Illusion I,spotify:track:3YRCqOhFifThpSRFJ1VWFM,None,...,None,None,None,trackdone,remote,True,False,False,1.747257e+09,False
198803,2025-05-15T09:53:37Z,android,331560,BR,189.57.250.90,Enter Sandman - Remastered 2021,Metallica,Metallica,spotify:track:3DwQ7AH3xGD9h65ezslm6q,None,...,None,None,None,trackdone,trackdone,True,False,False,1.747302e+09,False
